In [3]:
import os
import csv
import pandas  as pd
import nltk
import gensim
from nltk.corpus import stopwords
from gensim import corpora,models,similarities
from gensim.models.doc2vec import TaggedDocument, Doc2Vec
import  difflib
import nltk.tokenize

In [7]:
df = pd.read_csv("../Data/posts.csv")

In [10]:
for i in range(df.__len__()):
    questions=str(df['body'][i]).lower()
    stop_words = set(stopwords.words('english'))
    preprocess=[]
    preprocess.append(nltk.tokenize.word_tokenize(questions))
    filtered_sentence = " ".join([w for w in preprocess[0] if not w in stop_words])
    tokens = nltk.wordpunct_tokenize(filtered_sentence)
    text = nltk.Text(tokens)
    words = " ".join([w.lower() for w in text if w.isalpha()])
    df['body'][i]=words
    
    questions=str(df['comments'][i]).lower()
    stop_words = set(stopwords.words('english'))
    preprocess=[]
    preprocess.append(nltk.tokenize.word_tokenize(questions))
    filtered_sentence = " ".join([w for w in preprocess[0] if not w in stop_words])
    tokens = nltk.wordpunct_tokenize(filtered_sentence)
    text = nltk.Text(tokens)
    words = " ".join([w.lower() for w in text if w.isalpha()])
    df['comments'][i]=words

C:\Users\0x6f736f646f\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
C:\Users\0x6f736f646f\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [14]:
df.to_csv("../Data/preprocess.csv")

In [12]:
texts = df.to_dict('records')
documents = [TaggedDocument(text['body'].split(), [text['comments']])  for text in texts]   ## Pass the 'questions' & 'answers'

model = gensim.models.Doc2Vec(vector_size=100, window=2, min_count=1, workers=11,alpha=0.025, min_alpha=0.025, epochs=20)
model.build_vocab(documents)
model.train(documents, epochs=model.iter, total_examples=model.corpus_count)

model.save("../doc2vec.model")

C:\Users\0x6f736f646f\AppData\Local\Programs\Python\Python37\lib\site-packages\gensim\models\base_any2vec.py:743: UserWarning: C extension not loaded, training will be slow. Install a C compiler and reinstall gensim for fast training.
  "C extension not loaded, training will be slow. "
C:\Users\0x6f736f646f\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  


In [ ]:
from scipy import spatial

In [24]:
class QABot:
    def __init__(self, data, model):
        self.df = data
        self.model = model
        
    def avg_feature_vector(self, sentence, model, num_features, index2word_set):
        #FUNCTION TO AVERAGE  ALL  OUR FEATURE  VECTORS  IN EVERY DOCUMENT 
        words = sentence.split()
        feature_vec = np.zeros((num_features,), dtype='float32') #Initialize empty array
        n_words = 0
        for word in words:
            if word in index2word_set:
                n_words += 1
                feature_vec = np.add(feature_vec, model[word])
        if (n_words > 0):
            feature_vec = np.divide(feature_vec, n_words)
        return feature_vec

    def cosine_dist(self, user_asked):
        #COSINE  SIMILARITY  BETWEEN  VECTORS  AND  GIVE PERCENTAGE  OF MATCH  BETWEEN  COMMON QUESTIONS  

        index2word_set = set(self.model.wv.index2word)
        try:
            all_ratios = []
            questn_df = pd.DataFrame({})
            for i in range(self.df.__len__()):
                s1_afv = self.avg_feature_vector(user_asked, model=self.model, num_features=100,
                                                 index2word_set=index2word_set)
                s2_afv = self.avg_feature_vector(self.df['questions'][i], model=self.model, num_features=100,
                                                 index2word_set=index2word_set)
                all_ratios.append(1 - spatial.distance.cosine(s1_afv, s2_afv))
            questn_df = pd.DataFrame({"questions": list(self.df['questions']), "answers": list(self.df['answers']),
                                      "type": list(self.df['type']),
                                      "ratios": all_ratios})
            final_ratio = questn_df.sort_values('ratios', ascending=False)

            if final_ratio.empty:

                return 'sorry  didnt  understand  your question'
            else:

                return final_ratio.head(5)
        except:
            return 'sorry  didnt  understand  your question'

    def preprocedd_user_inpt(self, user_asked):
        #THIS FUNCTION IS USED  TO  PREPROCESS  OUR  USER INPUT BY REMOVING  STOPWORDS  LIKE "IS" ,"AND" ,"THE" , TOKENIZE  THE  USER INPUT .
        a = user_asked.lower()
        stop_words = set(stopwords.words('english'))
        preprocess = []
        preprocess.append(nltk.tokenize.word_tokenize(a))  ### tokenize  user input
        filtered_sentence = " ".join([w for w in preprocess[0] if not w in stop_words])  ##  remove stopwords
        words = filtered_sentence
        return words


In [30]:
def main():

    data = pd.read_csv("../Data/preprocess.csv")
    model = Doc2Vec.load("../doc2vec.model")
    data['body'].fillna("nothing", inplace=True)
    Bot = QABot(data, model)
    print('write your question below : ')  #### initial  message  shown to users
    user_ask = input()
    user_inpt = int(input('Press 1 to know the answer to your question  | Press 2 to know top 5 probabable  matching answers'))

    if (user_inpt == 1):  ######### If user input  ==1  ,  it will  show  you  the answer to the question
        userIp = Bot.preprocedd_user_inpt(int(user_ask))  ######## Calling the  preprocesser  function from class Bot
        predict_question = Bot.cosine_dist(user_ask).iloc[0].answers  #### Calling the cosine_dist function from class Bot
        print(predict_question)
    elif (user_inpt == 2):  ######### If user input  ==2  ,  it will  show  you  the top 5  matching  questions with probabilities .
        userIp = Bot.preprocedd_user_inpt(user_ask)  ######## Calling the  preprocesser  function from class Bot
        predict_question = Bot.cosine_dist(int(user_ask))[
            ['questions', 'answers', 'ratios']]  #### Calling the cosine_dist function from class Bot
        print(predict_question)

    else:
        print('please type 1 or 2 for answer')



In [33]:
if __name__ == '__main__':
    main()

write your question below : 


 Hello
Press 1 to know the answer to your question  | Press 2 to know top 5 probabable  matching answers 5


please type 1 or 2 for answer
